In [ ]:
#키워드를 추출한 데이터가 본 ipynb 파일과 같은 디렉토리에 있어야함

# load data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt

from collections import Counter
from itertools import islice

In [3]:
df_1 = pd.read_csv("./keyword/1st_period_rep.csv")
df_2 = pd.read_csv("./keyword/2nd_period_rep.csv")
df_3 = pd.read_csv("./keyword/3rd_period_rep.csv")

## 함수 정의
* size(): 시기마다 날짜별로 총 기사 수를 추출
* is_vaccine(): 백신이 포함된 keyword가 있는 날짜의 기사만 추출
* adj_and_merge(): 날짜 / 백신 언급 수 / 기사 수 / 백신 언급 비율 / 시기를 column값으로 가지는 dataframe 반환

-----------------------

백신 언급 비율 -> 백신 언급 수 % 기사수

기사 하나당 키워드 하나씩을 뽑았으니 해당 날짜의 기사수==언급된 키워드 수

각 날짜마다 크롤링한 기사 수가 다르기 때문에 백신이 얼마나 언급됐는지 절대값이 아니라 비율로 구하는 것이 정확

In [4]:
def size(df):
    grouped = df['Keyword'].groupby(df['publish'])
    return grouped.size()

In [5]:
def is_vaccine(df):
    is_vaccine = df['Keyword'] == '백신'
    vaccine = df[is_vaccine]
    return vaccine

In [6]:
def adj_and_merge(or_df, vacc_df, num):
    new_df = vacc_df.reset_index().rename(columns={"index": "publish"})
    sz = or_df.groupby('publish').size().reset_index(name='size')
    new_df = pd.merge(new_df,sz, on="publish")
    new_df.insert(loc=3, column='adj_vacc', value=new_df['vacc_mention']/new_df['size'])
    if num == 2:
        new_df['period'] = ["period_2" for i in range(new_df.shape[0])]
    else:
        new_df['period'] = ["period_3" for i in range(new_df.shape[0])]
    return new_df

In [7]:
print("각 시기별 날짜 당 기사 수\n")
print("대유행 1기: {}\n".format(size(df_1)))
print("대유행 2기: {}\n".format(size(df_2)))
print("대유행 3기: {}".format(size(df_3)))

각 시기별 날짜 당 기사 수

대유행 1기: publish
2020.03.03.    1386
2020.03.04.    1287
2020.03.05.    1881
2020.03.06.    1584
2020.03.07.    1089
2020.03.08.    1485
2020.03.09.    1287
2020.03.10.    1782
Name: Keyword, dtype: int64

대유행 2기: publish
2020.08.15.    1485
2020.08.16.    1386
2020.08.17.    1881
2020.08.18.    1881
2020.08.19.     990
2020.08.20.    1782
2020.08.21.    1782
2020.08.22.    1683
2020.08.23.    1584
2020.08.24.    1386
2020.08.25.    1485
2020.08.26.    1338
2020.08.27.     693
2020.08.28.    1188
2020.08.29.    1782
2020.08.30.    2079
2020.08.31.    1287
2020.09.01.    1287
2020.09.02.     990
Name: Keyword, dtype: int64

대유행 3기: publish
2020.12.15.    1782
2020.12.16.    1089
2020.12.17.    1584
2020.12.18.    1782
2020.12.19.    1782
2020.12.20.    1980
2020.12.21.    1881
2020.12.22.    1683
2020.12.23.    1683
2020.12.24.    2079
2020.12.25.    2178
2020.12.26.    1782
2020.12.27.    1980
2020.12.28.    1386
2020.12.29.    1386
2020.12.30.     792
2020.12.31.    13

In [8]:
vaccine_1 = is_vaccine(df_1)
vaccine_1.head()
#1번째 시기에는 백신이 keyword인 기사가 없음
#이후에 1번째 시기는 제외하고 진행

,title,publish,Keyword


In [9]:
vaccine_2 = is_vaccine(df_2)
vaccine_2.head()

,title,publish,Keyword
48,"러시아 보건부 ""코로나19 백신 1차분 생산돼""",2020.08.15.,백신
96,"러시아 보건부 ""코로나19 백신 1차분 생산돼""",2020.08.15.,백신
138,"러시아 보건부 ""코로나19 백신 1차분 생산돼""",2020.08.15.,백신
188,"러시아 보건부 ""코로나19 백신 1차분 생산돼""",2020.08.15.,백신
213,"러시아 보건부 ""코로나19 백신 1차분 생산돼""",2020.08.15.,백신


In [10]:
vaccine_3 = is_vaccine(df_3)
vaccine_3.head()

,title,publish,Keyword
1,해외여행이 그리워… 국민 70% “코로나 백신 맞겠다”,2020.12.15.,백신
3,"유럽의약품청 ""화이자 코로나19 백신 회의 21일로 앞당겨""(종합)",2020.12.15.,백신
7,"로이터 ""유럽의약품청, 23일 코로나19 백신 승인 예정""",2020.12.15.,백신
20,"유럽의약품청, 화이자 코로나19 백신 조건부 승인 권고(종합)",2020.12.21.,백신
25,"""유럽의약품청, 화이자 코로나 백신 조건부 판매 승인 권고""",2020.12.21.,백신


In [11]:
#인덱스(날짜) 순서대로 세주기
series_2= vaccine_2["publish"].value_counts().sort_index()
series_3= vaccine_3["publish"].value_counts().sort_index()

#dataframe으로
new_df_2= series_2.to_frame(name='vacc_mention')
new_df_3= series_3.to_frame(name='vacc_mention')

In [12]:
vacc_df_2 = adj_and_merge(df_2, new_df_2, 2)
vacc_df_3 = adj_and_merge(df_3, new_df_3, 3)

In [13]:
vacc_df_2.head()

,publish,vacc_mention,size,adj_vacc,period
0,2020.08.15.,99,1485,0.066667,period_2
1,2020.08.16.,2,1386,0.001443,period_2
2,2020.08.17.,1,1881,0.000532,period_2
3,2020.08.19.,1,990,0.001010,period_2
4,2020.08.25.,688,1485,0.463300,period_2


In [14]:
vacc_df_3.head()

,publish,vacc_mention,size,adj_vacc,period
0,2020.12.15.,592,1782,0.332211,period_3
1,2020.12.16.,99,1089,0.090909,period_3
2,2020.12.17.,297,1584,0.187500,period_3
3,2020.12.18.,395,1782,0.221661,period_3
4,2020.12.19.,3,1782,0.001684,period_3


In [15]:
#합쳐서 하나의 dataframe으로 만들어주기
df_concat = pd.concat([vacc_df_2, vacc_df_3])
df_concat

,publish,vacc_mention,size,adj_vacc,period
0,2020.08.15.,99,1485,0.066667,period_2
1,2020.08.16.,2,1386,0.001443,period_2
2,2020.08.17.,1,1881,0.000532,period_2
3,2020.08.19.,1,990,0.001010,period_2
4,2020.08.25.,688,1485,0.463300,period_2
5,2020.08.30.,297,2079,0.142857,period_2
0,2020.12.15.,592,1782,0.332211,period_3
1,2020.12.16.,99,1089,0.090909,period_3
2,2020.12.17.,297,1584,0.187500,period_3
3,2020.12.18.,395,1782,0.221661,period_3


In [16]:
df_concat.to_csv("./keyword/vaccine_data.csv", index=False, encoding="utf-8-sig")